In [1]:
import cv2
import mediapipe as mp
import numpy as np

import os
import pickle

# Preprocessing des datas 

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def get_landmarks(image_rgb, pose_model): 
    '''
    Récupère les landmarks d'une image donnée en RGB via un modèle déjà chargé.
    Return: un numpy array (33,4) ou None si rien n'est trouvé.
    '''

    results = pose_model.process(image_rgb)
    
    if results.pose_landmarks:

        pose_np = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark])
        return pose_np

    return None

In [3]:
def get_dataset(folder_path, path_save, pickle_name="dataset.pkl"): 
    ''' récupère les landmarks d'images dans un dossier via la fonction get_landmarks(image)

    folder_path : dossier de récuparation des images
    folder_dest : folder to save the numpy into à pickle 

    return numpy dataset folder  '''

    mp_pose = mp.solutions.pose
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:


        data_list = []
        fichiers = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        path_save_pickle = os.path.join(path_save, pickle_name)

        for name in fichiers:
            img_path = os.path.join(folder_path, name)
            img = cv2.imread(img_path)
            
            if img is not None:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                landmarks_vector = get_landmarks(img_rgb, pose) #un vecteur avec les coordonées
                
                if landmarks_vector is not None:
                    data_list.append(landmarks_vector.flatten())
                else:
                    print(f"Ignorée : {name}")
        
        dataset = np.array(data_list)

        with open(path_save_pickle, 'wb') as f:
                    pickle.dump(dataset, f)

    return dataset

## attributs target  

In [4]:
def get_target(dataset, class_int): 
    ''''récupère la target du dataset'''

    nb_images = dataset.shape[0]
    class_id = class_int
    target = np.full(nb_images, class_id)

    return target 


# Dataset des postures 

In [5]:
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
    folder_path= r'C:\Users\Emeline_PRO\Documents\Modules\Module_ACV\partie_3\data\Arbre'
    path_save = r'C:\Users\Emeline_PRO\Documents\Modules\Module_ACV\partie_3\datasets'
    get_dataset(folder_path, path_save, pickle_name="dataset_arbre.pkl")

    folder_path= r'C:\Users\Emeline_PRO\Documents\Modules\Module_ACV\partie_3\data\Cobra'
    get_dataset(folder_path, path_save, pickle_name="dataset_cobra.pkl")

    folder_path= r'C:\Users\Emeline_PRO\Documents\Modules\Module_ACV\partie_3\data\downdog'
    get_dataset(folder_path, path_save, pickle_name="dataset_downdog.pkl")
    

Ignorée : 00000104.jpg
Ignorée : 00000202.jpg
Ignorée : image18.jpeg
Ignorée : images133.jpg
Ignorée : images216.jpg
Ignorée : images54.jpg
Ignorée : 1_364.jpg
Ignorée : 3_287.jpg
Ignorée : File26.png
Ignorée : File57.png
Ignorée : File59.png
Ignorée : File63.png
Ignorée : File68.jpeg
Ignorée : image10.jpeg
Ignorée : images12.jpg
Ignorée : PXL_20251217_103746696.MP.jpg
Ignorée : Twisting-Cobra-pose.jpg
Ignorée : downdog (18).jpg
Ignorée : downdog (39).jpg
Ignorée : downdog (5).jpg
Ignorée : downdog (52).jpg
Ignorée : downdog (68).jpg
Ignorée : downdog (85).jpg
Ignorée : downdog (92).jpg
Ignorée : PXL_20251217_104256162.MP.jpg


In [12]:
chemin_du_fichier = r'C:\Users\Emeline_PRO\Documents\Modules\Module_ACV\partie_3\datasets\dataset_downdog.pkl'

with open(chemin_du_fichier, 'rb') as f:
    dataset = pickle.load(f)

In [13]:
dataset.shape

(142, 132)

In [15]:
get_target(dataset, 1).shape

(142,)